# Setup

In [ ]:
pip install requests pandas duckdb datetime

# Functions

In [40]:
import requests
import pandas as pd
import duckdb
import datetime

# Schritt 1: Daten von der RKI API abrufen
def fetch_covid_data():
    url = "https://api.corona-zahlen.org/states/history/incidence"
    response = requests.get(url)
    response.raise_for_status()  # Stellt sicher, dass ein Fehler ausgelöst wird, wenn die Anfrage fehlschlägt
    data = response.json()
    return data

# Schritt 2: Daten in eine Datenbank erfassen
def save_data_to_duckdb(data, db_path="covid_data.duckdb"):
    # Überprüfen, ob die 'data' und 'states' Schlüssel vorhanden sind
    if 'data' not in data:
        raise KeyError("Key 'data' not found in the API response")
    
    # Extrahieren und normalisieren der Daten
    states_data = data['data']
    records = []
    for state, state_data in states_data.items():
        history = state_data.get('history', [])
        for record in history:
            record['state'] = state
            records.append(record)
    
    df = pd.DataFrame(records)
    
    # Überprüfen, ob die Spalte 'weekIncidence' vorhanden ist
    if 'weekIncidence' not in df.columns:
        raise KeyError("Column 'weekIncidence' not found in the data")
    
    # Spalte 'weekIncidence' in 'incidence' umbenennen für Konsistenz
    df.rename(columns={'weekIncidence': 'incidence'}, inplace=True)
    
    # Verbindung zu DuckDB herstellen und Daten speichern
    conn = duckdb.connect(database=db_path, read_only=False)
    conn.execute("CREATE TABLE IF NOT EXISTS covid_incidence (state VARCHAR, date DATE, incidence DOUBLE)")
    conn.register('df_view', df)
    conn.execute("INSERT INTO covid_incidence SELECT state, date, incidence FROM df_view")
    conn.close()

# Schritt 3: Daten normalisieren
def normalize_data(db_path="covid_data.duckdb"):
    conn = duckdb.connect(database=db_path, read_only=False)
    conn.execute("""
    CREATE TABLE IF NOT EXISTS normalized_covid_incidence AS 
    SELECT 
        state, 
        date, 
        incidence, 
        ROW_NUMBER() OVER (PARTITION BY state ORDER BY date) AS row_num 
    FROM covid_incidence
    """)
    conn.close()

# Schritt 4: Daten transformieren und bereitstellen (SCD Type 2)
def transform_data_scd2(db_path="covid_data.duckdb"):
    conn = duckdb.connect(database=db_path, read_only=False)
    conn.execute("""
    CREATE TABLE IF NOT EXISTS scd_covid_incidence (
        state VARCHAR, 
        date DATE, 
        incidence DOUBLE, 
        valid_from DATE, 
        valid_to DATE, 
        is_current BOOLEAN
    )
    """)
    # Beende vorherige Einträge
    conn.execute("""
    UPDATE scd_covid_incidence
    SET valid_to = current_date - INTERVAL '1' DAY,
        is_current = FALSE
    WHERE is_current = TRUE
    """)
    # Füge neue Einträge hinzu
    conn.execute("""
    INSERT INTO scd_covid_incidence (state, date, incidence, valid_from, valid_to, is_current)
    SELECT 
        state, 
        date, 
        incidence, 
        current_date AS valid_from, 
        '9999-12-31' AS valid_to, 
        TRUE AS is_current
    FROM normalized_covid_incidence
    """)
    conn.close()

# Schritt 5: Tests implementieren
def run_tests(db_path="covid_data.duckdb"):
    conn = duckdb.connect(database=db_path, read_only=False)
    # Teste, ob Daten vorhanden sind
    results = conn.execute("SELECT state, COUNT(*) AS cnt FROM covid_incidence GROUP BY state HAVING cnt > 0").fetchall()
    assert len(results) > 0, "Test failed: No data found in covid_incidence table"
    print("Test passed: Data found in covid_incidence table")
    
    # Teste, ob die SCD2 Transformation funktioniert hat
    results = conn.execute("SELECT state, COUNT(*) AS cnt FROM scd_covid_incidence WHERE is_current = TRUE GROUP BY state HAVING cnt > 0").fetchall()
    assert len(results) > 0, "Test failed: No current data found in scd_covid_incidence table"
    print("Test passed: Current data found in scd_covid_incidence table")
    conn.close()

# Hauptfunktion
def main():
    data = fetch_covid_data()
    save_data_to_duckdb(data)
    normalize_data()
    transform_data_scd2()
    run_tests()



# Main

In [41]:
if __name__ == "__main__":
    main()

Test passed: Data found in covid_incidence table
Test passed: Current data found in scd_covid_incidence table


# Ad-Hoc Analysis

In [57]:
conn = duckdb.connect(database="covid_data.duckdb", read_only=False)
conn.sql("""
    SELECT
        state
        ,date
        ,incidence
        ,valid_from
        ,valid_to
        ,is_current
    FROM scd_covid_incidence
    WHERE is_current = TRUE
    """).show()

┌─────────┬────────────┬────────────────────┬────────────┬────────────┬────────────┐
│  state  │    date    │     incidence      │ valid_from │  valid_to  │ is_current │
│ varchar │    date    │       double       │    date    │    date    │  boolean   │
├─────────┼────────────┼────────────────────┼────────────┼────────────┼────────────┤
│ SH      │ 2020-01-07 │                0.0 │ 2024-05-21 │ 9999-12-31 │ true       │
│ SH      │ 2020-01-08 │                0.0 │ 2024-05-21 │ 9999-12-31 │ true       │
│ SH      │ 2020-01-09 │                0.0 │ 2024-05-21 │ 9999-12-31 │ true       │
│ SH      │ 2020-01-10 │                0.0 │ 2024-05-21 │ 9999-12-31 │ true       │
│ SH      │ 2020-01-11 │                0.0 │ 2024-05-21 │ 9999-12-31 │ true       │
│ SH      │ 2020-01-12 │                0.0 │ 2024-05-21 │ 9999-12-31 │ true       │
│ SH      │ 2020-01-13 │                0.0 │ 2024-05-21 │ 9999-12-31 │ true       │
│ SH      │ 2020-01-14 │                0.0 │ 2024-05-21 │ 9999-1

# Backup

In [ ]:
# crontab -e
# 0 0 * * * /usr/bin/python3 ./showcase2.ipynb

1